In [1]:
from ipfml import processing
from ipfml import utils
from ipfml import metrics
from PIL import Image
from scipy import signal
from skimage import color
import scipy.stats as stats
import seaborn as sns
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
data_folder = "../fichiersSVD_light"

# SVD analysis on zones of Synthesis Images 

## Utils functions definition

In [3]:
def compute_images_path(dict_data):
    scene = dict_data['name']
    prefix = dict_data['prefix']
    indices = dict_data['indices']
    
    images_path = []
    for index in indices:
        path = os.path.join(data_folder, os.path.join(scene, prefix + index + ".png"))
        print(path)
        images_path.append(path)
    return images_path

In [4]:
def get_images_zones(dict_data, images_path):
    
    zones_indices = dict_data['zones']
    zones_img = []
    
    for path in images_path:
        img = Image.open(path)
        zones = processing.divide_in_blocks(img, (200, 200))
        
        zones_list = []
        
        for id_zone in zones_indices:
            zones_list.append(zones[id_zone])
            
        zones_img.append(zones_list)
        
    return zones_img

In [5]:
def display_sv_data(dict_data, zones_data, interval, _norm=False):
    
    scene_name = dict_data['name']
    image_indices = dict_data['indices']
    zones_indices = dict_data['zones']
    colors = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']
    
    plt.figure(figsize=(25, 20))
    
    sv_data = []
    begin, end = interval
    for id_img, zones in enumerate(zones_data):
        
        for id_zone, zone in enumerate(zones):
            U, s, V = processing.get_LAB_L_SVD(zone)
        
            data = s[begin:end]
            
            if _norm:
                data = utils.normalize_arr(data)
                
            plt.plot(data, 
                     color=colors[id_zone], 
                     label='Zone ' + str(zones_indices[id_zone]) + ' of ' + scene_name + '_' + str(image_indices[id_img]))
            
    plt.legend(fontsize=18)
    plt.show()

In [6]:
# Useful function

def get_highest_values(arr, n):
    return np.array(arr).argsort()[-n:][::-1]

def get_lowest_values(arr, n):
    return np.array(arr).argsort()[::-1][-n:][::-1]

## Scenes zones data

In [7]:
# start 00020 - ref 00900 - step 10
dict_appart = {'name': 'Appart1opt02', 
               'prefix': 'appartAopt_', 
               'indices': ["00020", "00200", "00900"],
               'zones': [3, 6]}

# start 00050 - ref 01200 - step 10
dict_cuisine = {'name': 'Cuisine01', 
               'prefix': 'cuisine01_', 
               'indices': ["00050", "00400", "01200"],
               'zones': [3, 6]}

# start 00020 - ref 00950 - step 10
dict_sdb_c = {'name': 'SdbCentre', 
               'prefix': 'SdB2_', 
               'indices': ["00020", "00400", "00950"],
               'zones': [3, 6]}

# start 00020 - ref 00950 - step 10
dict_sdb_d = {'name': 'SdbDroite', 
               'prefix': 'SdB2_D_', 
               'indices': ["00020", "00400", "00950"],
               'zones': [2, 3, 10, 13]}

In [8]:
current_dict = dict_appart
interval = (30, 200)

In [9]:
images_path = compute_images_path(current_dict)
zones_data = get_images_zones(current_dict, images_path)

../fichiersSVD_light/Appart1opt02/appartAopt_00020.png
../fichiersSVD_light/Appart1opt02/appartAopt_00200.png
../fichiersSVD_light/Appart1opt02/appartAopt_00900.png


In [12]:
first_image = zones_data[0][1]
entropy_contribution_data = []

sv = processing.get_LAB_L_SVD_s(first_image)
sv = utils.normalize_arr(sv)
entropy = utils.get_entropy(sv)

for i in range(200):
    entropy_without_column = utils.get_entropy_without_i(sv, i)
    entropy_contribution_column = entropy - entropy_without_column
    entropy_contribution_data.append(entropy_contribution_column)

In [13]:
get_highest_values(entropy_contribution_data, 100)

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100])

In [14]:
get_lowest_values(entropy_contribution_data, 100)

array([  0, 199, 198, 197, 196, 195, 194, 193, 192, 191, 190, 189, 188,
       187, 186, 185, 184, 183, 182, 181, 180, 179, 178, 177, 176, 175,
       174, 173, 172, 171, 170, 169, 168, 167, 166, 165, 164, 163, 162,
       161, 160, 159, 158, 157, 156, 155, 154, 153, 152, 151, 150, 149,
       148, 147, 146, 145, 144, 143, 142, 141, 140, 139, 138, 137, 136,
       135, 134, 133, 132, 131, 130, 129, 128, 127, 126, 125, 124, 123,
       122, 121, 120, 119, 118, 117, 116, 115, 114, 113, 112, 111, 110,
       109, 108, 107, 106, 105, 104, 103, 102, 101])